In [1]:
%run 02_retrieval.ipynb

d:\code\multi-modal-rag-llmops\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FAISS index loaded.
BM25 retrieved: 10 chunks
Semantic retrieved: 10 chunks
Combined unique: 13

----- Chunk 1 -----
Source: Microsoft-2024-Annual-Report.pdf
Page: None
supporting and investing in our cloud-based services, including datacenter operations; designing, manufacturing, marketing, 
and selling our other products and services; and income taxes.  
Highlights from fiscal year 2024 compared with fiscal year 2023 included:  
• 
Microsoft Cloud revenue increas ...

----- Chunk 2 -----
Source: Microsoft-2024-Annual-Report.pdf
Page: None
32 
Intelligent Cloud  
Revenue increased $17.5 billion or 20%.  
• 
Server products and cloud services revenue increased $17.8 billion or 22% driven by Azure and other cloud 
services. Azure and other cloud services revenue grew 30% driven by growth in our consumption-based 
services. Server produc ...

----- Chunk 3 -----
Source: Microsoft-2024-Annual-Report.pdf
Page: None
37,884   
31%  
More Personal Computing 
 
19,309   
16,450   
17%  
  
  


In [2]:
from sentence_transformers import CrossEncoder

# bge reranker
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

d:\code\multi-modal-rag-llmops\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rk\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
def rerank_documents(query, docs, top_k=10):
    if not docs:
        return []

    # build pairs (query, doc_text)
    pairs = [(query, d.page_content) for d in docs]

    # run cross encoder scoring
    scores = reranker.predict(pairs)

    # attach scores to docs
    for doc, score in zip(docs, scores):
        doc.metadata["rerank_score"] = float(score)

    # sort by score (descending)
    ranked = sorted(docs, key=lambda x: x.metadata["rerank_score"], reverse=True)

    return ranked[:top_k]

In [6]:
def show_reranked(docs, top_n=10):
    for i, d in enumerate(docs[:top_n]):
        print(f"--- Rank {i+1} | Score: {d.metadata['rerank_score']:.4f} ---")
        print(d.page_content[:300], "...")
        print("Metadata:", d.metadata)
        print("\n")


In [7]:
retrieved = retrieve_candidates(query)    # BM25 + semantic + dedupe + optional self-query

reranked = rerank_documents(query, retrieved, top_k=10)

show_reranked(reranked)


BM25 retrieved: 10 chunks
Semantic retrieved: 10 chunks
Combined unique: 13
--- Rank 1 | Score: 7.6230 ---
supporting and investing in our cloud-based services, including datacenter operations; designing, manufacturing, marketing, 
and selling our other products and services; and income taxes.  
Highlights from fiscal year 2024 compared with fiscal year 2023 included:  
• 
Microsoft Cloud revenue increas ...
Metadata: {'source': 'Microsoft-2024-Annual-Report.pdf', 'company': 'Microsoft', 'year': 2024, 'doctype': 'Annual-Report', 'page': None, 'rerank_score': 7.6230316162109375}


--- Rank 2 | Score: 5.4904 ---
to our hardware portfolio, and costs related to lease consolidation activities. Refer to the Non-GAAP Financial Measures 
section below for a reconciliation of our financial results reported in accordance with GAAP to non-GAAP financial results.  
Fiscal Year 2024 Compared with Fiscal Year 2023  
Re ...
Metadata: {'source': 'Microsoft-2024-Annual-Report.pdf', 'company': 'Microsof